In [3]:
import numpy as np
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm

from torch.utils.data import DataLoader

batch_size = 500
window_size = 21

In [4]:
df = pd.read_csv('/kaggle/input/renmindaily/RenMin_Daily.csv')
df

,id,date,title,content
0,20210101-01-01,20210101,中俄两国元首互致新年贺电 中俄两国总理互致新年贺电,新华社北京12月31日电 2020年12月31日，国家主席习近平和俄罗斯总统普京互致新年...
1,20210101-01-02,20210101,国家主席习近平发表二〇二一年新年贺词,■ 2020年是极不平凡的一年。面对突如其来的新冠肺炎疫情，我们以人民至上、生命至上诠释...
2,20210101-01-03,20210101,艰难方显勇毅，磨砺始得玉成 ——习近平主席二〇二一年新年贺词启示录①,新故相推，日生不滞。在风雨兼程中，我们告别2020年，迎来2021年。\n “艰难方显...
3,20210101-01-04,20210101,全国政协举行新年茶话会 习近平发表重要讲话李克强栗战书王沪宁赵乐际韩正王岐山出席 汪洋主持,新华社北京12月31日电 中国人民政治协商会议全国委员会12月31日上午在全国政协礼堂举...
4,20210101-02-01,20210101,在全国政协新年茶话会上的讲话 （2020年12月31日）,同志们，朋友们：\n 在风雨兼程中，我们即将送别2020年，迎来2021年新年。很高兴在这...
...,...,...,...,...
106329,20250517-08-02,20250517,数字化，让文物迸发丰沛的生命力,\t\t\t\t\t\t\t许 蒙\n\t\t\t\t\t\t\t\t《人民日报》（202...
106330,20250517-08-03,20250517,看中国也看世界,\t\t\t\t\t\t\t王 樾\n\t\t\t\t\t\t\t\t《人民日报》（202...
106331,20250517-08-04,20250517,小县城有所“大学校”,\t\t\t\t\t\t\t邓建胜\n\t\t\t\t\t\t\t\t《人民日报》（2025...
106332,20250517-08-05,20250517,文创，萃取文物的美学价值,\t\t\t\t\t\t\t胡继禹\n\t\t\t\t\t\t\t\t《人民日报》（2025...


In [4]:
import csv

def csv_to_text(csv_path, txt_path, separator=' '):
    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        all_text = []
        for row in reader:
            # 将每行的所有单元格用分隔符连接
            row_text = separator.join(str(cell) for cell in row)
            all_text.append(row_text)
        
        # 将所有行合并为一个字符串
        final_text = '\n'.join(all_text)
        
        # 写入文本文件
        with open(txt_path, 'w', encoding='utf-8') as txtfile:
            txtfile.write(final_text)

# 使用示例
csv_to_text('/kaggle/input/renmindaily/RenMin_Daily.csv', 'output.txt', separator=' ')

In [5]:
df = pd.read_text('/kaggle/working/output.txt')
df

NameError: name 'pd' is not defined

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def get_batch(split):
    # 选择训练或验证数据集
    data = train_data if split == 'train' else val_data

    # 动态从数据集中选择位置索引
    if len(data) < block_size:
        # 如果数据不足，填充到block_size长度
        padding = torch.zeros(block_size + 1 - len(data), dtype=torch.long)
        data = torch.cat([data, padding])
    
    # 生成随机索引
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x.to(device),y.to(device)

class Head(nn.Module):
    """单头 self-attention """
    def __init__(self, n_embd):
        super().__init__()
        self.key = nn.Linear(n_embd, n_embd, bias=False)
        self.query = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(n_embd, n_embd, bias=False)

    def forward(self, input_x):
        B, T, C = input_x.shape

        k = self.key(input_x)
        q = self.query(input_x)
        v = self.value(input_x)

        wei = q @ k.transpose(-2,-1) * C ** -0.5
        T = wei.shape[-1]
        tril = torch.tril(torch.ones(T,T, device=device))
        wei = wei.masked_fill(tril == 0, float('-inf'))
        wei = wei.softmax(dim=-1)

        out = wei @ v
        return out


class BingramLanguageModel(nn.Module):
    
    def __init__(self, block_size, vocab_size, n_embd):
        super().__init__()
        # 每个token都直接从Embedding中查询对应的logits值 以进行下一个token的推理
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # 位置编码
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # one head self-attention
        self.sa_head = Head(n_embd)
        # larg model forward
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B,T = idx.shape
        
        # idx值和targets值都是整型张量 (B,T)
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.sa_head(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx指当前语料集(B,T)中的索引
        for _ in range(max_new_tokens):
            # 限定索引列的取值范围
            idx_cond = idx[:, -block_size:]
            # 推理
            logits, loss = self(idx_cond)
            # 只提取最后一个时间步的结果
            logits = logits[:, -1, :]  # (B,C)
            # 通过softmax转换为概率值
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # 随机采样
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
            # 把采样的索引追加在当前解码序列末尾
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

if __name__ == '__main__':

    # 模型训练数据集
    block_size = 8
    batch_size = 32
    max_iter = 10000
    learn_rate = 1e-3
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    n_embd = 32
    eval_interval = 500
    eval_iters = 200

    
    with open('/kaggle/working/output.txt') as f:
        text = f.read()

    # 字典、编码器(函数)、解码器(函数)
    chars = sorted(list(set(text)))
    vocab_size = len(chars)
    stoi = {ch:i for i,ch in enumerate(chars)}  #str_to_index
    itos = {i:ch for i,ch in enumerate(chars)}  #index_to_str

    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])

    # 文本转换token index
    data = torch.tensor(encode(text), dtype=torch.long)

    # 拆分数据集
    n = int(len(data) * .9)
    train_data = data[:n]
    val_data = data[n:]

    # 模型训练
    model = BingramLanguageModel(block_size, vocab_size, n_embd)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learn_rate)

    @torch.no_grad()
    def estimate_loss():
        out = {}
        model.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                logits, loss = model(X, Y)
                losses[k] = loss.item()
                print(f"Batch{k}: loss = {loss.item():.4f}")
            out[split] = losses.mean()
        model.train()
        return out

    for iter in range(max_iter):

        if iter % eval_interval == 0:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

        # 批次样本
        xb, yb = get_batch('train')

        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # 模型生成
    idx = torch.zeros((1,1), dtype=torch.long, device=device)
    print(decode(model.generate(idx, max_new_tokens=500)[0].tolist())) 


Batch0: loss = 9.0275
Batch1: loss = 8.9897
Batch2: loss = 9.0359
Batch3: loss = 9.0041
Batch4: loss = 9.0099
Batch5: loss = 8.9970
Batch6: loss = 9.0272
Batch7: loss = 9.0276
Batch8: loss = 9.0214
Batch9: loss = 9.0408
Batch10: loss = 9.0187
Batch11: loss = 8.9905
Batch12: loss = 8.9607
Batch13: loss = 9.0132
Batch14: loss = 9.0373
Batch15: loss = 8.9967
Batch16: loss = 8.9650
Batch17: loss = 9.0063
Batch18: loss = 9.0186
Batch19: loss = 9.0261
Batch20: loss = 9.0158
Batch21: loss = 8.9993
Batch22: loss = 8.9803
Batch23: loss = 8.9986
Batch24: loss = 8.9857
Batch25: loss = 9.0145
Batch26: loss = 9.0004
Batch27: loss = 8.9998
Batch28: loss = 8.9775
Batch29: loss = 8.9986
Batch30: loss = 8.9861
Batch31: loss = 8.9944
Batch32: loss = 9.0111
Batch33: loss = 9.0049
Batch34: loss = 9.0326
Batch35: loss = 9.0078
Batch36: loss = 8.9935
Batch37: loss = 8.9993
Batch38: loss = 9.0050
Batch39: loss = 8.9690
Batch40: loss = 9.0056
Batch41: loss = 9.0229
Batch42: loss = 9.0213
Batch43: loss = 9.019

In [25]:
model = BingramLanguageModel(block_size, vocab_size, n_embd)
model.to(device)

BingramLanguageModel(
  (token_embedding_table): Embedding(1, 32)
  (position_embedding_table): Embedding(8, 32)
  (sa_head): Head(
    (key): Linear(in_features=32, out_features=32, bias=False)
    (query): Linear(in_features=32, out_features=32, bias=False)
    (value): Linear(in_features=32, out_features=32, bias=False)
  )
  (lm_head): Linear(in_features=32, out_features=1, bias=True)
)